In [1]:
from os.path import join
from pathlib import Path
import os,sys
import os.path as op
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
import mne
mne.viz.set_3d_backend("pyvista")
from scipy.stats import sem
from scipy.io import savemat, loadmat
from mne.datasets import fetch_fsaverage
from mne.beamformer import make_lcmv, apply_lcmv, apply_lcmv_epochs
import numpy as np

from defintions import ROOT__DIR

Using pyvistaqt 3d backend.



# Cortical Parcellation

Using the Desikan-Killiany Atlas, comprising of 68 cortical regions bilaterally. 

- Paper: https://surfer.nmr.mgh.harvard.edu/ftp/articles/desikan06-parcellation.pdf

- FreeSurfer: https://surfer.nmr.mgh.harvard.edu/fswiki/CorticalParcellation

In [2]:
# ANATOMICAL PARCELLATION (LABELS)
# the full list of cortical labels
mylabel_names = ['bankssts-lh',
                'bankssts-rh',
                'caudalanteriorcingulate-lh',
                'caudalanteriorcingulate-rh',
                'caudalmiddlefrontal-lh',
                'caudalmiddlefrontal-rh',
                'cuneus-lh',
                'cuneus-rh',
                'entorhinal-lh',
                'entorhinal-rh',
                'frontalpole-lh',
                'frontalpole-rh',
                'fusiform-lh',
                'fusiform-rh',
                'inferiorparietal-lh',
                'inferiorparietal-rh',
                'inferiortemporal-lh',
                'inferiortemporal-rh',
                'insula-lh',
                'insula-rh',
                'isthmuscingulate-lh',
                'isthmuscingulate-rh',
                'lateraloccipital-lh',
                'lateraloccipital-rh',
                'lateralorbitofrontal-lh',
                'lateralorbitofrontal-rh',
                'lingual-lh',
                'lingual-rh',
                'medialorbitofrontal-lh',
                'medialorbitofrontal-rh',
                'middletemporal-lh',
                'middletemporal-rh',
                'paracentral-lh',
                'paracentral-rh',
                'parahippocampal-lh',
                'parahippocampal-rh',
                'parsopercularis-lh',
                'parsopercularis-rh',
                'parsorbitalis-lh',
                'parsorbitalis-rh',
                'parstriangularis-lh',
                'parstriangularis-rh',
                'pericalcarine-lh',
                'pericalcarine-rh',
                'postcentral-lh',
                'postcentral-rh',
                'posteriorcingulate-lh',
                'posteriorcingulate-rh',
                'precentral-lh',
                'precentral-rh',
                'precuneus-lh',
                'precuneus-rh',
                'rostralanteriorcingulate-lh',
                'rostralanteriorcingulate-rh',
                'rostralmiddlefrontal-lh',
                'rostralmiddlefrontal-rh',
                'superiorfrontal-lh',
                'superiorfrontal-rh',
                'superiorparietal-lh',
                'superiorparietal-rh',
                'superiortemporal-lh',
                'superiortemporal-rh',
                'supramarginal-lh',
                'supramarginal-rh',
                'temporalpole-lh',
                'temporalpole-rh',
                'transversetemporal-lh',
                'transversetemporal-rh']

# the region of interest (ROI) from the motor cortex. 
roi_labels = ['postcentral-lh',
            'postcentral-rh',
            'precentral-lh',
            'precentral-rh']


# Routine

## Forward and Inverse models

In [5]:
fname = join(ROOT__DIR, 'data/epochs-epo.fif')
cond = 'left' # 'right'


epochs = mne.read_epochs(fname=fname, preload=True) # load the MI/ME EEG epochs
epochs.resample(256.0) # down-sample to 256 Hz. 
epochs = epochs[cond]

# Apply projection, as suggested by MNE-Python
epochs = epochs.set_eeg_reference(projection=True).apply_proj()

# Extract the Data Covariance and Noise Covariance Matrix
data_tmin, data_tmax = (0.5, 2.0) # change to 4s for full-window length
noise_tmin, noise_tmax = (-1.0, 0.0)
crop_tmin, crop_tmax = (-1, 2.1)

# Computing the covariance matrices
data_cov = mne.compute_covariance(epochs, tmin=data_tmin, tmax=data_tmax, method='empirical', verbose=False)
noise_cov = mne.compute_covariance(epochs, tmin=noise_tmin, tmax=noise_tmax, method='empirical', verbose=False)

# Crop epochs in the time of interest
cropped_epochs = epochs.crop(crop_tmin, crop_tmax) # Leave extra 100ms for smearing effects after wavelet

del epochs # save memory

########## MAKE FORWARD SOLUTION #################################
subject = 'fsaverage'
trans = 'fsaverage'  # MNE has a built-in fsaverage transformation
fs_dir = fetch_fsaverage(verbose=True)
src = os.path.join(fs_dir, 'bem', 'fsaverage-ico-5-src.fif')
bem = os.path.join(fs_dir, 'bem', 'fsaverage-5120-5120-5120-bem-sol.fif') # average BEM model

fwd = mne.make_forward_solution(cropped_epochs.info, trans=trans, src=src, bem=bem, eeg=True, mindist=5.0, n_jobs=-1, verbose=False) # standard params

leadfield = fwd['sol']['data'] # get the leadfield matrix (nSensors-x-nSources)
print("Leadfield size : %d sensors x %d dipoles" % leadfield.shape)

fwd_fname = '' # file name to save the forward solution
# mne.write_forward_solution(fwd_fname, fwd, overwrite=True, verbose=False) # uncomment to save the forward model

src = fwd['src']

########## INVERSE MODEL (LCMV Filters) ##########################
filters = make_lcmv(cropped_epochs.info, fwd, data_cov, reg=0.05,
                    noise_cov=noise_cov, pick_ori='max-power',
                    weight_norm='unit-noise-gain', rank=None, verbose=False)

Reading /home/oem/Documents/HBI-motor_imagery/HBI-motor_imagery/data/epochs-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    4000.00 ms
        0 CTF compensation matrices available
Not setting metadata
49 matching events found
No baseline correction applied
0 projection items activated
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
0 files missing from root.txt in /home/oem/mne_data/MNE-fsaverage-data
0 files missing from bem.txt in /home/oem/mne_data/MNE-fsaverage-data/fsaverage
Leadfield size : 64 sensors x 61452 dipoles


## Time-courses in ROIs

- This may take a while!

In [7]:
fs_dir = fetch_fsaverage(verbose=True)
subjects_dir = os.path.dirname(fs_dir)

# Get time-courses for all sources
time_courses = {}
for label in roi_labels:

    labels_parc = mne.read_labels_from_annot(subject, parc='aparc', subjects_dir=subjects_dir, regexp=label, verbose=False)[0]

    stcall = apply_lcmv_epochs(epochs=cropped_epochs, filters=filters, return_generator=True, verbose=False)

    time_courses[label] = []

    assert label == labels_parc.name
    
    for stc_epoch in stcall:

        pca_anat = mne.extract_label_time_course(stc_epoch, labels_parc, src, mode='pca_flip', verbose=False)[0]
        # flip the pca so that the max power between tmin and tmax is positive
        pca_anat *= np.sign(pca_anat[np.argmax(np.abs(pca_anat))])  
        # Append time course to list for this label         
        time_courses[label].append(pca_anat)

0 files missing from root.txt in /home/oem/mne_data/MNE-fsaverage-data
0 files missing from bem.txt in /home/oem/mne_data/MNE-fsaverage-data/fsaverage
Processing epoch : 1
Processing epoch : 2
Processing epoch : 3
Processing epoch : 4
Processing epoch : 5
Processing epoch : 6
Processing epoch : 7
Processing epoch : 8
Processing epoch : 9
Processing epoch : 10
Processing epoch : 11
Processing epoch : 12
Processing epoch : 13
Processing epoch : 14
Processing epoch : 15
Processing epoch : 16
Processing epoch : 17
Processing epoch : 18
Processing epoch : 19
Processing epoch : 20
Processing epoch : 21
Processing epoch : 22
Processing epoch : 23
Processing epoch : 24
[done]
Processing epoch : 1
Processing epoch : 2
Processing epoch : 3
Processing epoch : 4
Processing epoch : 5
Processing epoch : 6
Processing epoch : 7
Processing epoch : 8
Processing epoch : 9
Processing epoch : 10
Processing epoch : 11
Processing epoch : 12
Processing epoch : 13
Processing epoch : 14
Processing epoch : 15
Pr

## Time-Frequency Reconstruction

- Apply baseline correction

In [8]:
# Get a time vector from MNE
tf_epochs, _ = mne.time_frequency.tfr_morlet(cropped_epochs, freqs=np.arange(8, 31, 1), output='power', n_cycles=5, use_fft=True, decim=3, n_jobs=1)
time_vector = tf_epochs.times

In [24]:
# Define a zeros matrix of shape nLabels, 2, nTime, where 2 is alpha and beta. 
srate = cropped_epochs.info['sfreq']

tf_time_courses = np.zeros(shape=(len(roi_labels), 2, time_vector.shape[-1]))

for label_idx, label in enumerate(roi_labels):

    data = np.asarray(time_courses[label]) # Extract data from ROI into a numpy array

    # TF decomposition using wavelet convolution
    cued_tf = mne.time_frequency.tfr_array_morlet(data[:, None, :], sfreq=srate, freqs=np.arange(8, 31, 1), output='power', n_cycles=5, use_fft=True, decim=3, n_jobs=1)

    cued_tf = cued_tf.mean(axis=0)

    # apply baseline correction (average across trials is computed in the argument)
    baseline_cued_tf = mne.baseline.rescale(data=cued_tf, times=time_vector, baseline=(-.500, 0), mode='zscore')[0, :, :]
    alpha = baseline_cued_tf[:6, :].mean(axis=0) # average across alpha (8-13 Hz)
    beta = baseline_cued_tf[6:, :].mean(axis=0) # average across beta
    cued_tf = np.concatenate((alpha[None, :], beta[None, :]), axis=0)
    tf_time_courses[label_idx, :, :] = cued_tf

    # save numpy array if relevant
    # np.save(fname, full_tf_time_courses_)

Applying baseline correction (mode: zscore)
Applying baseline correction (mode: zscore)
Applying baseline correction (mode: zscore)
Applying baseline correction (mode: zscore)
